In [1]:

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 

from sklearn.metrics import classification_report
from transformers import AutoTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score


# the dataset downloaded directly from hf hub: https://huggingface.co/datasets/xnli/viewer/ar
#issues when loading directly from hf

df1 = pd.read_parquet('xnli_ar_dataset/train-00000-of-00001.parquet')
df2 = pd.read_parquet('xnli_ar_dataset/validation-00000-of-00001.parquet')
df3 = pd.read_parquet('xnli_ar_dataset/test-00000-of-00001.parquet')

df = pd.concat([df3, df2], ignore_index=True)  # combine val and test ds for validation
dataset_train = Dataset.from_pandas(df1)
dataset_validation = Dataset.from_pandas(df)

# Get the unique labels from the 'label' column
unique_labels = set(dataset_train['label'])
classes_num = len(unique_labels)

print(f'Unique Labels: {unique_labels}')
print(f'Number of Classes: {classes_num}')

# url = 'faisalq/bert-base-arabic-wordpiece'
url = 'faisalq/bert-base-arabic-senpiece'
# url = 'faisalq/bert-base-arabic-bbpe'




tokenizer = AutoTokenizer.from_pretrained(url)
model = BertForSequenceClassification.from_pretrained(url,
                                                      num_labels=classes_num).to('cuda')                                                 
                                                     



max_length = 128

def preprocess_function(examples):
    return tokenizer(examples['premise'], examples['hypothesis'], truncation=True, padding="max_length",
                    max_length=max_length)


dataset_train = dataset_train.map(preprocess_function, batched=True)
dataset_validation = dataset_validation.map(preprocess_function, batched=True)

# Define the compute_metrics function for evaluation



2024-02-06 18:14:55.639497: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-06 18:14:55.663089: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-06 18:14:56.092875: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Unique Labels: {0, 1, 2}
Number of Classes: 3


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at faisalq/bert-base-arabic-senpiece and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

In [2]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)    
    acc = accuracy_score(labels, predictions)        
    f1 = f1_score(labels, predictions, average='macro')      
    return {'accuracy': acc, 'f1_score': f1}


epochs = 5
save_steps = 10000 #save checkpoint every 10000 steps
batch_size = 256


training_args = TrainingArguments(
    output_dir = 'bert_sp/',
    overwrite_output_dir=True,
    num_train_epochs = epochs,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    save_steps = save_steps,
    save_total_limit = 2, 
    fp16=True,
    learning_rate = 5e-5,  # 5e-5 is the default
    # weight_decay=0.01,
    logging_steps = 1000, #50_000
    evaluation_strategy = 'steps',
    eval_steps = 500
    
)




# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_validation,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()


/home/ffq/miniconda3/envs/p1/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1 Score
500,No log,0.711027,0.692800,0.693093
1000,0.774000,0.698402,0.698400,0.698177
1500,0.774000,0.666126,0.715600,0.715370
2000,0.622700,0.641313,0.740933,0.740978
2500,0.622700,0.640308,0.738400,0.738339
3000,0.565000,0.620809,0.742800,0.743750
3500,0.565000,0.687721,0.731200,0.732152
4000,0.453300,0.687046,0.736400,0.736065
4500,0.453300,0.658012,0.745333,0.745429
5000,0.406600,0.762722,0.733867,0.733277


{'eval_loss': 0.8293928503990173,
 'eval_accuracy': 0.7402666666666666,
 'eval_f1_score': 0.7403023960870224,
 'eval_runtime': 3.1744,
 'eval_samples_per_second': 2362.686,
 'eval_steps_per_second': 9.451,
 'epoch': 5.0}